In [1]:
import os
import numpy as np
import json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Flatten
from sklearn.model_selection import train_test_split

In [2]:
# Get the current working directory
CURRENT_DIRECTORY = os.getcwd()
# Get the parent directory
PARENT_DIRECTORY = os.path.dirname(CURRENT_DIRECTORY)
PARENT_DIRECTORY = os.path.dirname(PARENT_DIRECTORY)

# Open the config file and load its content into a dictionary
config_file = open(PARENT_DIRECTORY + '\\config\\config.json')
CONFIG_DATA = json.load(config_file)

# Close the file after loading the data
config_file.close()

In [3]:
WINDOW_SIZE = CONFIG_DATA['WINDOW_SIZE']
NUMBER_OF_FEATURES = CONFIG_DATA["NUMBER_OF_FEATURES"]
NUM_EPOCHS = CONFIG_DATA["NUMBER_OF_EPOCHS"]
CLASSICATIONS = CONFIG_DATA['CLASSES_MOTIONTYPE']

ALL_X_TRAINING_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-X-TRAIN-CLASSIFY-MOTIONTYPE-LSIDESTEPS-V0.npy'
ALL_Y_TRAINING_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-Y-TRAIN-CLASSIFY-MOTIONTYPE-LSIDESTEPS-V0.npy'

ALL_X_TEST_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-X-TEST-CLASSIFY-MOTIONTYPE-LSIDESTEPS-V0.npy'
ALL_Y_TEST_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-Y-TEST-CLASSIFY-MOTIONTYPE-LSIDESTEPS-V0.npy'

MOTION_MODEL_SAVE_PATH =  PARENT_DIRECTORY + '\\NeuralNetwork\\models\\standard-backprop-motiontype-lsidesteps-2.h5'

In [15]:
ALL_X_TRAIN_DATA = np.load(ALL_X_TRAINING_DATA_PATH)
ALL_Y_TRAIN_DATA = np.load(ALL_Y_TRAINING_DATA_PATH)

ALL_X_TEST_DATA = np.load(ALL_X_TEST_DATA_PATH)
ALL_Y_TEST_DATA = np.load(ALL_Y_TEST_DATA_PATH)

ALL_X_TRAIN_DATA = ALL_X_TRAIN_DATA[:, 1]
ALL_X_TEST_DATA = ALL_X_TEST_DATA[:, 1]

ALL_X_COMBINED_DATA = np.concatenate((ALL_X_TRAIN_DATA, ALL_X_TEST_DATA))
ALL_Y_COMBINED_DATA = np.concatenate((ALL_Y_TRAIN_DATA, ALL_Y_TEST_DATA))


ALL_X_TRAIN_DATA, ALL_X_TEST_DATA, ALL_Y_TRAIN_DATA, ALL_Y_TEST_DATA = train_test_split(ALL_X_COMBINED_DATA, ALL_Y_COMBINED_DATA, test_size=0.2, random_state=42)


print(ALL_X_TRAIN_DATA[:100])
print(ALL_Y_TRAIN_DATA[:100])

print(ALL_X_TRAIN_DATA.shape)
print(ALL_Y_TRAIN_DATA.shape)

print(ALL_X_TEST_DATA.shape)
print(ALL_Y_TEST_DATA.shape)

[-14. -12.  -7.  -7. -12. -19.   3.  -7. -11.  -9.   2.   5.  -6.  -6.
  -6. -15.  -7. -20.  -2.   5.  -8.  -7. -10. -12.  -9.  -9.  -2.  -8.
 -21.   3.  -7. -15. -12. -10. -14.  -2.  -5.  -3.  -6.  -9.  -5. -14.
  -9.  -9. -17.  -4.  -3. -22. -15.  -9. -10.  -3. -12. -14.  -8. -14.
  -2.   0.  -6.  -3. -17.  -9.   0.  -5.  -6.   3. -15.  -8.  -5.  -7.
  -7. -15. -15.  -6.  -7. -13. -14.  -3. -13. -13. -16. -20.  -8. -14.
  -3.  -7.  -7.  -5.   0. -12. -11. -21.  -7.  -8. -10.  -2.  -5.   3.
 -18. -20.]
[0 0 1 1 0 0 1 0 0 0 1 0 1 0 0 0 1 0 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 1
 1 1 0 1 0 1 1 0 1 1 0 0 0 0 1 0 0 1 0 1 1 0 0 0 1 1 0 1 1 0 0 0 1 1 0 0 1
 1 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 1 0 1 0 0]
(86005,)
(86005,)
(21502,)
(21502,)


In [5]:
# Generate random permutation indices
random_indices_1 = np.random.permutation(len(ALL_X_TRAIN_DATA))


# Shuffle the input features and target labels using the random indices
shuffled_X_TRAIN_DATA = ALL_X_TRAIN_DATA[random_indices_1]
shuffled_Y_TRAIN_DATA = ALL_Y_TRAIN_DATA[random_indices_1]

In [6]:
# Generate random permutation indices
random_indices_2 = np.random.permutation(len(ALL_X_TEST_DATA))

# Shuffle the input features and target labels using the random indices
shuffled_X_TEST_DATA = ALL_X_TEST_DATA[random_indices_2]
shuffled_Y_TEST_DATA = ALL_Y_TEST_DATA[random_indices_2]

In [7]:
output_activation_function = 'sigmoid' if len(CLASSICATIONS) == 2 else 'softmax'
loss_algorithm = 'binary_crossentropy' if len(CLASSICATIONS) == 2 else 'sparse_categorical_crossentropy'
output_size = 1 if len(CLASSICATIONS) == 2 else len(CLASSICATIONS)

In [8]:
model2_motion = Sequential()
model2_motion.add(InputLayer((1)))
model2_motion.add(Dense(8, activation='relu'))
model2_motion.add(Dense(4, activation='relu'))
model2_motion.add(Dense(output_size, activation=output_activation_function))

model2_motion.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 16        
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Compile the model
model2_motion.compile(optimizer='adam', loss=loss_algorithm, metrics=['acc'])

history2_motion = model2_motion.fit(shuffled_X_TRAIN_DATA, shuffled_Y_TRAIN_DATA, validation_data=(shuffled_X_TEST_DATA, shuffled_Y_TEST_DATA), epochs=NUM_EPOCHS)

# Save the model to a file
model2_motion.save(MOTION_MODEL_SAVE_PATH)

Epoch 1/12
2688/2688 [==============================] - 5s 2ms/step - loss: 0.5012 - acc: 0.6987 - val_loss: 0.4404 - val_acc: 0.7599
Epoch 2/12
2688/2688 [==============================] - 4s 1ms/step - loss: 0.4342 - acc: 0.7610 - val_loss: 0.4294 - val_acc: 0.7599
Epoch 3/12
2688/2688 [==============================] - 4s 2ms/step - loss: 0.4281 - acc: 0.7620 - val_loss: 0.4224 - val_acc: 0.7626
Epoch 4/12
2688/2688 [==============================] - 4s 2ms/step - loss: 0.4253 - acc: 0.7622 - val_loss: 0.4215 - val_acc: 0.7626
Epoch 5/12
2688/2688 [==============================] - 5s 2ms/step - loss: 0.4250 - acc: 0.7626 - val_loss: 0.4226 - val_acc: 0.7626
Epoch 6/12
2688/2688 [==============================] - 5s 2ms/step - loss: 0.4246 - acc: 0.7620 - val_loss: 0.4217 - val_acc: 0.7626
Epoch 7/12
2688/2688 [==============================] - 4s 2ms/step - loss: 0.4246 - acc: 0.7614 - val_loss: 0.4238 - val_acc: 0.7626
Epoch 8/12
2688/2688 [==============================] - 4s 2ms